<a href="https://colab.research.google.com/github/takafumi12/gigwork_quest_orange_team/blob/hanada/gigworkquest_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# digitsデータを使うためにimportする
from torchvision.datasets import CIFAR10
import torchvision
from torchvision import transforms as transforms

In [24]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cpu


In [4]:
# import zipfile

# with zipfile.ZipFile('/content/drive/MyDrive/Data/gigwork_quest/train.zip') as existing_zip:
#     existing_zip.extractall('/content/drive/MyDrive/Data/gigwork_quest')

# with zipfile.ZipFile('/content/drive/MyDrive/Data/gigwork_quest/test.zip') as existing_zip:
#     existing_zip.extractall('/content/drive/MyDrive/Data/gigwork_quest')

# 画像の読み込み

In [25]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
df = pd.read_csv('/content/drive/MyDrive/Data/gigwork_quest/train_master.tsv',sep='\t')
y = np.array(df["label_id"])

In [46]:
import glob
import os.path as osp
import torch.utils.data as data
from torchvision import models, transforms
from PIL import Image


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # transforms.RandomResizedCrop(
                #     resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                # transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)


def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = PATH + "MyDrive/Data/gigwork_quest/"
    target_path = osp.join(rootpath+phase+'/*.png')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list


class HymenopteraDataset(data.Dataset):
    """
    画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    """

    def __init__(self, file_list, y, transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.y = y #画像の正解ラベル
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        return img_transformed, y[index]


In [47]:
from torch.utils.data.dataset import Subset

# 1.3節で作成したクラスを同じフォルダにあるmake_dataset_dataloader.pyに記載して使用
#from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

#import sys
#sys.path.append('/content/drive/My Drive/Colab/my_project/my_modules')

# 画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
# test_list = make_datapath_list(phase="test")

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
    file_list=train_list, y=y, transform=ImageTransform(size, mean, std), phase='train')

# test_dataset = HymenopteraDataset(
#     file_list=test_list, transform=ImageTransform(size, mean, std), phase='test')

# print(len(train_dataset)) # 231
# print(len(val_dataset)) # 59
# print(len(test_dataset)) # 213

# # DataLoaderを作成する
# batch_size = 32

# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader = torch.utils.data.DataLoader(
#     val_dataset, batch_size=batch_size, shuffle=False)

# test_dataloader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=1, shuffle=False)

# # 辞書オブジェクトにまとめる
# dataloaders_dict = {"train2": train_dataloader, "val2": val_dataloader}

/content/drive/MyDrive/Data/gigwork_quest/train/*.png


In [48]:
#train_testに分割
n_samples = len(train_dataset)
train_size = int(len(train_dataset) * 0.8)
val_size = n_samples - train_size

trainset, testset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [49]:
print(len(trainset),len(testset))

38100 9525


In [35]:
classes = tuple([i for i in range(20)])
classes

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)

In [50]:
train_loader = DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = DataLoader(testset, batch_size=16, shuffle=True)

## 転移学習
---
今回はVGG16を利用しました。

In [51]:
# Enter your code here
model_ft2 = models.vgg16(pretrained=True)

In [52]:
# パラメーターを学習させないようにしている
# Enter your code here
for param in model_ft2.parameters():
    param.require_grad = False

In [53]:
# 最後の一層だけ書き換えて、最後の一層だけ学習させる
# Enter your code here
model_ft2.fc = nn.Linear(in_features=512, out_features=len(classes), bias=True)

In [54]:
loss_func = nn.CrossEntropyLoss()
print(loss_func)

CrossEntropyLoss()


In [55]:
optimizer = optim.SGD(model_ft2.parameters(), lr=0.001)

In [56]:
model_ft2 = model_ft2.to(device)

In [57]:
param = {
    'trainloader': train_loader,
    'valloader': test_loader,
    'net': model_ft2,
    'optimizer': optimizer,
    'lossfunc': loss_func,
    'epochs': 5,
    'device': device
    }

In [58]:
def training(trainloader, valloader, net, optimizer, lossfunc, epochs, device):
    train_loss_track = []
    test_loss_track = []

    for epoch in range(1, epochs+1):
        running_loss = 0
        running_train_loss = 0
        running_test_loss = 0

        # 学習モード
        net.train()
        for step, batch  in enumerate(trainloader, 1):
            # device = "cuda"の場合、GPUにデータを転送する
            xx, yy = batch[0].to(device), batch[1].to(device)
            # 最後に計算した各パラメーターの勾配を初期化する
            optimizer.zero_grad()
            # フォワード計算を行う
            y_pred = net(xx)
            # 誤差関数を使ってギャップの計測
            loss = lossfunc(y_pred, yy)
            # 誤差逆伝播法を使って自動微分
            loss.backward()
            # パラメーターを更新
            optimizer.step()
            # 学習データを使って損失を計算
            running_loss += loss.item()
            running_train_loss += loss.item()

            # 100 iterationごとにlossを出力
            if step % 100 == 0:
                print('[{:d} epoch, {:5d} iter] train/loss: {}'.format(epoch, step, running_loss / 100))
                running_loss = 0.0
        # エポックが終了したら平均損失を計算
        train_loss_track.append(running_train_loss / len(train_loader))

        # 評価（evaluation）モード
        net.eval()
        # 勾配計算用のパラメータを保存しない
        with torch.no_grad():
            for step, batch in enumerate(valloader, 1):
                # device = "cuda"の場合、GPUにデータを転送する
                xx_test, yy_test = batch[0].to(device), batch[1].to(device)
                # 予測値を計算
                y_pred = net(xx_test)
                # 誤差関数を使ってギャップの計測
                test_loss = lossfunc(y_pred, yy_test)
                # テストデータを使って損失を計算
                running_test_loss += test_loss.item()
        # 誤差をトラッキング
        test_loss_track.append(running_test_loss / len(valloader))
        print('val/loss: {}'.format(running_test_loss / len(valloader)))
    return net, train_loss_track, test_loss_track

In [59]:
model_ft2, train_loss_track, test_loss_track = training(**param)

KeyboardInterrupt: ignored

## 交差エントロピー誤差の推移

In [ ]:
plt.plot(train_loss_track)

In [ ]:
plt.plot(test_loss_track)

In [ ]:
def calculate_accuracy(testloader, net):
    true = 0
    total = 0

    all_labels = np.array([])
    all_preds = np.array([])

    net.eval()
    with torch.no_grad():
        for test_xx, test_yy in testloader:
            
            # device = "cuda"の場合、GPUにデータを転送する
            test_xx = test_xx.to(device)
            test_yy = test_yy.to(device)

            outputs = net(test_xx)
            _, predicted = torch.max(outputs.data, 1)
            
            all_labels = np.append(all_labels, test_yy.cpu().data.numpy())
            all_preds = np.append(all_preds, predicted.cpu().numpy())
            
            total += test_yy.size(0)
            true += (predicted == test_yy).sum().item()
    print('Accuracy: {:.2f} %'.format(100 * float(true/total)))

    return all_labels, all_preds

In [ ]:
all_labels, all_preds = calculate_accuracy(test_loader, model_ft2)

## testデータで混同行列を作成

In [ ]:
def plot_conf_mat(all_labels, all_preds):
    labels = np.unique(all_labels)
    cm = confusion_matrix(all_labels, all_preds, labels=labels)
    cm_labeled = pd.DataFrame(cm, columns=labels, index=labels)
    return cm_labeled

plot_conf_mat(all_labels, all_preds)